### Türme von Hanoi [(siehe hier)](https://de.wikipedia.org/wiki/T%C3%BCrme_von_Hanoi)
Ein Scheibenstapel soll verschoben werden. Dabei sind folgende Regeln einzuhalten:
- Es darf jeweils nur eine Scheibe verschoben werden.
- Auf eine Scheibe darf nur eine kleinere Scheibe gelegt werden.
- Es dürfen max. 3 Stapel vorhanden sein.

Wir programmieren eine einfache, aber ausbaufähige Variante dieses Spiels.  
- Verschoben wird ein Stabel mit 4 Scheiben,
  repräsentiert durch die Zahlen 0, 1, 2 und 3. 
  Je grösser die Zahl,
  desto grösser die Scheibe.
- Eine absteigend sortierte Liste `[3, 2, 1, 0]` entspricht einem Stapel.
  Die letzte Zahl entspricht der obersten und kleinsten Scheibe.

### Der Kern des Spiels
Der Spielzustand wird in der Variable `stacks` gespeichert.
Diese enthält eine Liste mit 3 Listen, den Stapeln.
Die Funktion `new_game()` startet ein neues Spiel, indem die Liste `stacks` modifiziert wird, so dass sie die 3 Stapel  
`[3, 2, 1, 0]`, `[]` und `[]`  
enthält.

Die Funktion `move_disk(src, dst)` verschiebt eine Scheibe von Stapel `src` (source) auf den Stapel `dst` (destination), falls möglich (**besser**: falls regelkonform).  
Die Funktion  `new_game()` und `move_disk(src, dst)` geben zudem den aktuellen Spielzustand aus, indem 
Funktion `show_stacks(stacks)` aufgerufen wird.

In [ ]:
ndisks = 4
stacks = []


def show_stacks(stacks):
    print(stacks)


def new_game():
    stack_1 = list(range(ndisks))[::-1]
    stacks[:] = [stack_1, [], []]
    show_stacks(stacks)


def move_disk(src, dst):
    if not stacks[src]:
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)
    show_stacks(stacks)

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)

### Bildliche Darstellung der Stapelkonfiguration
Statt einfach den Spielzustand `stacks` mit `print` auszugeben,
soll nun die Funktion `show_stacks(stacks)` eine bildliche Darstellung liefern.
Zu desem Zweck schreiben wir eine Funktion `make_drawStacks_cmd(stacks)`,
die einen String liefert, welcher unseren Zeichnungsautomaten ein entsprechendes Bild zeichnen lässt.

Wir zerlegen des Problem in kleinere Teilprobleme. Zuerst
erstellen wir  die Zeichungsanweisung für eine einzelne Scheibe,
dann für einen einzelnen Stapel und schliesslich für alle Stapel. 
Farbe und Abmessungen der Scheiben sowie die Stapelpositionen 
entnehmen wir dem Dict `config`.

In [ ]:
config = {
    'colors': ('brown', 'teal', 'blue', 'purple'),  # Farben der Scheiben
    'widths': (30, 50, 70, 90),  # Breite der Scheiben
    'height': 10,                # Hoehe der Scheiben
    'xpos': (50, 150, 250),  # x-Koordinaten der Stapelmitten
    'ypos': 100,  # y-Koordinate der Oberkante der untersten Scheibe
}

In [ ]:
def make_drawDisk_cmd(disk, col, row):
    cmds = []
    width = config['widths'][disk]
    height = config['height']
    color = config['colors'][disk]

    x = config['xpos'][col] - width/2
    y = config['ypos'] - row*height

    cmds.append(f'f{color};')
    cmds.append('u')
    cmds.append(f'g{x},{y};')
    cmds.append(f'R{width},{height};')

    return ''.join(cmds)

In [ ]:
make_drawDisk_cmd(2, 0, 1)

In [ ]:
def make_drawStack_cmd(stack, col):
    cmds = [make_drawDisk_cmd(disk, col, i) for i, disk in enumerate(stack)]
    cmd = ''.join(cmds)
    return cmd

In [ ]:
make_drawStack_cmd([3, 2, 1], 1)

In [ ]:
def make_drawStacks_cmd(stacks):
    cmds = [make_drawStack_cmd(stack, i) for i, stack in enumerate(stacks)]
    cmd = ''.join(cmds)
    return cmd

In [ ]:
stacks = [[2, 1], [3, 0], []]
make_drawStacks_cmd(stacks)

In [ ]:
import zeichnungsautomat as ZA

WIDTH = 300
HEIGHT = 200


def read(cmd, drawingBoard=None):
    if drawingBoard is None:
        _, drawingBoard = ZA.get_automat_and_canvas(width=WIDTH, height=HEIGHT)
    drawingBoard.automaton.read(cmd)
    return drawingBoard

In [ ]:
db = read('')
db

In [ ]:
disk1 = make_drawDisk_cmd(3, 0, 0)
disk2 = make_drawDisk_cmd(2, 0, 1)
disk3 = make_drawDisk_cmd(0, 2, 0)

In [ ]:
db = read(disk1, db)
db = read(disk2, db)
db = read(disk3, db)

In [ ]:
# show stacks ueberschreiben
def show_stacks(stacks):
    db.clear()
    cmd = make_drawStacks_cmd(stacks)
    db.automaton.read(cmd)

In [ ]:
db  # Drawingbord anzeigen

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)  # verschiebe Scheibe und zeichne neuen Spielzustand